# Necessary Imports

In [1]:
import numpy as np
import os
import glob
import tensorflow as tf
import argparse
import ecdl

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


# The arguments that we pass in

In [3]:
class myargs():
    def __init__(self):
        self.exp_id = 'efficientnet_round1'
        self.epochs = 10
        self.model = 'efficientnet'
        self.model_type = 'b0'
        self.image_dims = 256
        self.rotation_range = 0
        self.width_shift_range = 0.0
        self.height_shift_range = 0.0
        self.brightness_range = None
        self.zoom_range = 0.0
        self.horizontal_flip = False
        self.vertical_flip = False
        self.validation_split = 0.2
        self.batch_size = 10
        self.class_mode = 'categorical'

args = myargs()

# Data Generators

In [4]:
train_dir = '/workspaces/ecdl/elbow_data/SABINE_Training_Set'

base_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=args.rotation_range,
                                                                       width_shift_range=args.width_shift_range,
                                                                       height_shift_range=args.height_shift_range,
                                                                       brightness_range=args.brightness_range,
                                                                       zoom_range=args.zoom_range,
                                                                       horizontal_flip=args.horizontal_flip,
                                                                       vertical_flip=args.vertical_flip,
                                                                       rescale=1./255,
                                                                       validation_split = args.validation_split)



train_data_gen = base_image_generator.flow_from_directory(batch_size=args.batch_size,
                                                        target_size=(args.image_dims, args.image_dims),
                                                        directory=train_dir,
                                                        shuffle=True,
                                                        class_mode=args.class_mode,
                                                        subset='training')



val_data_gen = base_image_generator.flow_from_directory(batch_size=args.batch_size,
                                                        target_size=(args.image_dims, args.image_dims),
                                                        directory=train_dir,
                                                        shuffle=True,
                                                        class_mode=args.class_mode,
                                                        subset='validation')

Found 883 images belonging to 2 classes.
Found 219 images belonging to 2 classes.


In [5]:
sample = next(train_data_gen)

In [6]:
print(sample[0].shape)
print(sample[1].shape)
print(sample[0].shape[1:])

(10, 256, 256, 3)
(10, 2)
(256, 256, 3)


# Model

In [14]:
model = ecdl.models.tensorflow_efficientnet(model_name=args.model_type,
                                            classes=2,
                                            weights=None,
                                            include_top=True,
                                            input_shape=sample[0].shape[1:])                                      

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']
             )
model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
rescaling_3 (Rescaling)         (None, 256, 256, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
normalization_3 (Normalization) (None, 256, 256, 3)  7           rescaling_3[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 257, 257, 3)  0           normalization_3[0][0]            
_____________________________________________________________________________________

In [23]:
model_base = ecdl.models.tensorflow_efficientnet(model_name=args.model_type,                                                 
                                                 weights='imagenet',
                                                 include_top=False,
                                                 input_shape=sample[0].shape[1:])                                      

# model_base.trainable = False

# Rebuild top
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model_base.output)
x = tf.keras.layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred")(x)

# Compile
model = tf.keras.Model(model_base.input, outputs, name="EfficientNet")
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)
model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
rescaling_11 (Rescaling)        (None, 256, 256, 3)  0           input_12[0][0]                   
__________________________________________________________________________________________________
normalization_11 (Normalization (None, 256, 256, 3)  7           rescaling_11[0][0]               
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 257, 257, 3)  0           normalization_11[0][0]           
_______________________________________________________________________________________

In [32]:
from efficientnet.tfkeras import EfficientNetB0

efficient_net = EfficientNetB0(
        weights='imagenet',
        input_shape=(256, 256, 3),
        include_top=False,
#         pooling='max'
    )

# Basic
# x = tf.keras.layers.Dense(128, activation="relu", name="dense_1")(efficient_net.output)
# x = tf.keras.layers.Dense(128, activation="relu", name="dense_2")(x)
# outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred")(x)

# Rebuild top
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(efficient_net.output)
x = tf.keras.layers.BatchNormalization()(x)
top_dropout_rate = 0.2
x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred")(x)

model = tf.keras.Model(efficient_net.input, outputs, name="EfficientNet")

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 128, 128, 32) 864         input_19[0][0]                   
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 128, 128, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 128, 128, 32) 0           stem_bn[0][0]                    
_______________________________________________________________________________________

# Callbacks

In [8]:
exp_path = os.path.join('/workspaces/ecdl/experiments', args.exp_id)
checkpoint = tf.keras.callbacks.ModelCheckpoint(exp_path+'/weights.{epoch}-{val_accuracy:.4f}.hdf5', monitor='val_accuracy', verbose=1,save_best_only=True)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', verbose = 1, patience=5)
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(exp_path,'result.csv'))
delete_excess = ecdl.callbacks.delete_excess_callback(exp_folder=exp_path)
callbacks_list = [checkpoint, earlystopping, csv_logger, delete_excess]  

# Training

In [11]:
history = model.fit(
        train_data_gen,
        epochs = args.epochs,
        steps_per_epoch = len(train_data_gen),        
        validation_data = val_data_gen,
        verbose = 1,
        validation_steps = len(val_data_gen),
        callbacks=callbacks_list)

Epoch 1/10
89/89 [==============================] - ETA: 0s - loss: 3.0817 - accuracy: 0.5153
Epoch 00001: val_accuracy improved from -inf to 0.45205, saving model to /workspaces/ecdl/experiments/efficientnet_round1/weights.1-0.4521.hdf5
89/89 [==============================] - 12s 133ms/step - loss: 3.0817 - accuracy: 0.5153 - val_loss: 2.8355 - val_accuracy: 0.4521
Epoch 2/10
89/89 [==============================] - ETA: 0s - loss: 1.4166 - accuracy: 0.5266
Epoch 00002: val_accuracy improved from 0.45205 to 0.54795, saving model to /workspaces/ecdl/experiments/efficientnet_round1/weights.2-0.5479.hdf5
89/89 [==============================] - 11s 126ms/step - loss: 1.4166 - accuracy: 0.5266 - val_loss: 1.2199 - val_accuracy: 0.5479
Epoch 3/10
89/89 [==============================] - ETA: 0s - loss: 1.3084 - accuracy: 0.4575
Epoch 00003: val_accuracy did not improve from 0.54795
89/89 [==============================] - 11s 119ms/step - loss: 1.3084 - accuracy: 0.4575 - val_loss: 0.7315

In [24]:
history = model.fit(
        train_data_gen,
        epochs = args.epochs,
        steps_per_epoch = len(train_data_gen),        
        validation_data = val_data_gen,
        verbose = 1,
        validation_steps = len(val_data_gen),
        callbacks=callbacks_list)

Epoch 1/10
89/89 [==============================] - ETA: 0s - loss: 1.8541 - accuracy: 0.5096
Epoch 00001: val_accuracy did not improve from 0.54795
89/89 [==============================] - 11s 129ms/step - loss: 1.8541 - accuracy: 0.5096 - val_loss: 39003.9570 - val_accuracy: 0.5479
Epoch 2/10
89/89 [==============================] - ETA: 0s - loss: 1.7863 - accuracy: 0.4983
Epoch 00002: val_accuracy did not improve from 0.54795
89/89 [==============================] - 11s 119ms/step - loss: 1.7863 - accuracy: 0.4983 - val_loss: 12877.6348 - val_accuracy: 0.5479
Epoch 3/10
89/89 [==============================] - ETA: 0s - loss: 1.3649 - accuracy: 0.5051
Epoch 00003: val_accuracy did not improve from 0.54795
89/89 [==============================] - 11s 121ms/step - loss: 1.3649 - accuracy: 0.5051 - val_loss: 399.6337 - val_accuracy: 0.4521
Epoch 4/10
89/89 [==============================] - ETA: 0s - loss: 1.3693 - accuracy: 0.4813
Epoch 00004: val_accuracy did not improve from 0.5479

In [28]:
history = model.fit(
        train_data_gen,
        epochs = args.epochs,
        steps_per_epoch = len(train_data_gen),        
        validation_data = val_data_gen,
        verbose = 1,
        validation_steps = len(val_data_gen),
        callbacks=callbacks_list)

Epoch 1/10
89/89 [==============================] - ETA: 0s - loss: 1.0513 - accuracy: 0.6059
Epoch 00001: val_accuracy improved from 0.54795 to 0.63470, saving model to /workspaces/ecdl/experiments/efficientnet_round1/weights.1-0.6347.hdf5
89/89 [==============================] - 12s 133ms/step - loss: 1.0513 - accuracy: 0.6059 - val_loss: 0.6536 - val_accuracy: 0.6347
Epoch 2/10
89/89 [==============================] - ETA: 0s - loss: 0.5819 - accuracy: 0.7305
Epoch 00002: val_accuracy did not improve from 0.63470
89/89 [==============================] - 11s 120ms/step - loss: 0.5819 - accuracy: 0.7305 - val_loss: 0.6673 - val_accuracy: 0.5982
Epoch 3/10
89/89 [==============================] - ETA: 0s - loss: 0.5006 - accuracy: 0.7667
Epoch 00003: val_accuracy improved from 0.63470 to 0.65753, saving model to /workspaces/ecdl/experiments/efficientnet_round1/weights.3-0.6575.hdf5
89/89 [==============================] - 11s 125ms/step - loss: 0.5006 - accuracy: 0.7667 - val_loss: 0.8

In [33]:
history = model.fit(
        train_data_gen,
        epochs = args.epochs,
        steps_per_epoch = len(train_data_gen),        
        validation_data = val_data_gen,
        verbose = 1,
        validation_steps = len(val_data_gen),
        callbacks=callbacks_list)

Epoch 1/10
89/89 [==============================] - ETA: 0s - loss: 1.2475 - accuracy: 0.5719
Epoch 00001: val_accuracy did not improve from 0.74886
89/89 [==============================] - 11s 127ms/step - loss: 1.2475 - accuracy: 0.5719 - val_loss: 0.6891 - val_accuracy: 0.5388
Epoch 2/10
89/89 [==============================] - ETA: 0s - loss: 0.7668 - accuracy: 0.6478
Epoch 00002: val_accuracy did not improve from 0.74886
89/89 [==============================] - 11s 118ms/step - loss: 0.7668 - accuracy: 0.6478 - val_loss: 0.7358 - val_accuracy: 0.5479
Epoch 3/10
89/89 [==============================] - ETA: 0s - loss: 0.7958 - accuracy: 0.6580
Epoch 00003: val_accuracy did not improve from 0.74886
89/89 [==============================] - 11s 118ms/step - loss: 0.7958 - accuracy: 0.6580 - val_loss: 1.4896 - val_accuracy: 0.5936
Epoch 4/10
89/89 [==============================] - ETA: 0s - loss: 0.6728 - accuracy: 0.7191
Epoch 00004: val_accuracy did not improve from 0.74886
89/89 [=